In [9]:
import pandas as pd

In [10]:
import numpy as py

In [12]:
pip install faker


Note: you may need to restart the kernel to use updated packages.


In [13]:
from faker import Faker

In [14]:
import random

In [15]:
from datetime import date, timedelta

In [16]:
fake = Faker('es_ES')

In [5]:
NUM_REGISTROS = 500
FECHA_INICIO = date(2025, 1, 1)
DIAS_DE_MUESTREO = 30

In [6]:
# Definición de categorías fijas
ESTRATOS = ['CABA', 'PBA', 'ENTRE RIOS', 'SANTA FE']
SEXOS = ['masculino', 'femenino']
NIVELES_EDUCATIVOS = ['Primaria', 'Secundaria', 'Técnico', 'Universitario', 'Postgrado']
VOTOS = ['FUERZA PATRIA', 'LIBERTAD AVANZA', 'Voto en Blanco', 'Nulo', 'No Sabe/No Responde']
VOTOS_ANTERIORES = ['MASSA', 'MILEI', 'Voto en Blanco', 'No Votó', 'Nulo']

In [17]:
# 1. Generación de los datos
def generar_encuesta_falsa(num_registros):
    datos = []
    for i in range(1, num_registros + 1):
        # 1.1. Generar la Fecha (dentro de un rango de días)
        fecha_encuesta = FECHA_INICIO + timedelta(days=random.randint(0, DIAS_DE_MUESTREO))

        # 1.2. Generar Edad (entre 18 y 90)
        edad = fake.random_int(min=18, max=90)
        
        datos.append({
            'Fecha': fecha_encuesta.strftime('%Y-%m-%d'),
            'Encuesta': f'ENC-{fecha_encuesta.strftime("%Y%m%d")}-{i}', # ID único
            'Estrato': random.choice(ESTRATOS),
            'Sexo': random.choice(SEXOS),
            'Edad': edad,
            'Nivel Educativo': random.choice(NIVELES_EDUCATIVOS),
            'Cantidad de Integrantes en el Hogar': fake.random_int(min=1, max=7),
            'Imagen del Candidato': fake.random_int(min=0, max=100), # Puntaje de 0 a 100
            'Voto': random.choice(VOTOS),
            'Voto Anterior': random.choice(VOTOS_ANTERIORES)
        })
    return pd.DataFrame(datos)

In [23]:
# 2. Creación del DataFrame
df_encuesta = generar_encuesta_falsa(NUM_REGISTROS)

In [24]:
# 3. Conversión de tipos (necesario para el análisis posterior)
df_encuesta['Fecha'] = pd.to_datetime(df_encuesta['Fecha'])
df_encuesta['Imagen del Candidato'] = df_encuesta['Imagen del Candidato'].astype(int)

In [25]:
# 4. Mostrar una muestra de los datos generados
print(f"DataFrame generado con {len(df_encuesta)} registros.")
print("\n--- Muestra de los Datos Falsos ---")
print(df_encuesta.head())

DataFrame generado con 500 registros.

--- Muestra de los Datos Falsos ---
       Fecha        Encuesta     Estrato       Sexo  Edad Nivel Educativo  \
0 2025-01-09  ENC-20250109-1  ENTRE RIOS   femenino    44         Técnico   
1 2025-01-12  ENC-20250112-2  ENTRE RIOS  masculino    65       Postgrado   
2 2025-01-09  ENC-20250109-3    SANTA FE   femenino    78       Postgrado   
3 2025-01-22  ENC-20250122-4  ENTRE RIOS   femenino    27      Secundaria   
4 2025-01-01  ENC-20250101-5         PBA   femenino    27   Universitario   

   Cantidad de Integrantes en el Hogar  Imagen del Candidato  \
0                                    6                     2   
1                                    3                     0   
2                                    6                    88   
3                                    2                    80   
4                                    1                    93   

                  Voto Voto Anterior  
0  No Sabe/No Responde         MILEI  

In [27]:
# 2. Agrupación por Fecha y Estrato muestral
# Este será el DataFrame principal para la mayoría de los análisis
df_encuesta = df_encuesta.groupby(['Fecha', 'Estrato'])

In [29]:
conteo_sexo = df_encuesta['Sexo'].value_counts().unstack(fill_value=0)
print("\n--- Conteo de Casos por Sexo (Fecha y Estrato) ---")
print(conteo_sexo)


--- Conteo de Casos por Sexo (Fecha y Estrato) ---
Sexo                   femenino  masculino
Fecha      Estrato                        
2025-01-01 CABA               0          1
           ENTRE RIOS         3          3
           PBA                4          3
           SANTA FE           1          1
2025-01-02 CABA               2          3
...                         ...        ...
2025-01-30 SANTA FE           2          2
2025-01-31 CABA               1          3
           ENTRE RIOS         2          2
           PBA                4          0
           SANTA FE           2          1

[121 rows x 2 columns]


In [30]:
# Definir las categorías de edad
bins = [18, 26, 36, 46, 56, np.inf] # np.inf para el límite superior 'más de 56 años'
labels = ['1) 18-25 años', '2) 26-35 años', '3) 36-45 años', '4) 46-55 años', '5) más de 56 años']

NameError: name 'np' is not defined